In [1]:
import gc
import time
import os

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

from keras.models import Model
from keras.optimizers import Adam
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras import regularizers
from pathlib import Path

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sn

from tensorflow.python.client import device_lib


from sklearn.model_selection import cross_val_score

from matplotlib import pyplot as plt

import mne

from sklearn.model_selection import train_test_split

from tensorflow.keras.layers import Input, Flatten


In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"


In [3]:
tf.test.gpu_device_name()

''

### Loading edf

In [4]:
file = "..\dataverse_files\h01.edf"
edfs_path = "..\dataverse_files"
manifest_path = "..\dataverse_files\MANIFEST.txt"

In [5]:
def load_patients_data(edfs_path):
    raw_patients_data = []
    
    edfs_file_names = [f for f in os.listdir(edfs_path) if f.endswith('.edf')]
    
    for file_name in edfs_file_names:
        path = edfs_path + '\\' + file_name 
        raw_data = mne.io.read_raw_edf(path, preload=True, verbose=False)
        raw_patients_data.append(raw_data)

    return raw_patients_data

In [6]:
raw_patients_data = load_patients_data(edfs_path)

### Filtered EEG signals segmentation

In [7]:
def get_label(edf):
    patient_edf_file_name = edf.filenames[0].split('\\')[-1]
    isSick = patient_edf_file_name.lower().startswith('s')
    return int(isSick == True) # 1 - is sick, 0 is healthy

In [8]:
def get_min_max_duration_for_classes(print_durations=False):
    min_SZ_negative_duration = float("inf") # healthy
    min_SZ_positive_duration = float("inf") # sick

    max_SZ_negative_duration = 0 # healthy
    max_SZ_positive_duration = 0 # sick

    for edf in raw_patients_data:
        duration = edf.times[-1]

        if(get_label(edf) == 0):
            min_SZ_negative_duration = duration if duration < min_SZ_negative_duration else min_SZ_negative_duration
            max_SZ_negative_duration = duration if duration > max_SZ_negative_duration else max_SZ_negative_duration
        else:
            min_SZ_positive_duration = duration if duration < min_SZ_positive_duration else min_SZ_positive_duration
            max_SZ_positive_duration = duration if duration > max_SZ_positive_duration else max_SZ_positive_duration


    print('SZ_negative: min =', min_SZ_negative_duration, ', max =', max_SZ_negative_duration)
    print('SZ_positive: min =', min_SZ_positive_duration, ', max =', max_SZ_positive_duration)
    
    return min_SZ_negative_duration, min_SZ_positive_duration, max_SZ_negative_duration, max_SZ_positive_duration

In [9]:
def crop_raw_data_to_equalize_duration_per_class():
    print("Duration per class before cropping: ")
    min_dur_neg, min_dur_pos, *_ = get_min_max_duration_for_classes(True)
    
    for edf in raw_patients_data:
        duration = edf.times[-1]

        if(get_label(edf) == 0):
            if(duration > min_dur_neg):
                edf.crop(tmin=0, tmax=min_dur_neg, include_tmax=True)
        else:
            if(duration > min_dur_pos):
                edf.crop(tmin=0, tmax=min_dur_pos, include_tmax=True)
                
    print("\nDuration per class after cropping: ")

    get_min_max_duration_for_classes(True)


In [10]:
def print_info(epochs_num_per_patient, labels):
    print('\nEpochs number per patient: ', epochs_num_per_patient)
    
    class_SZ_positive = sum(labels) 
    class_SZ_negative= len(labels)-sum(labels)

    print('\nnegative: ', class_SZ_positive)
    print('positive: ', class_SZ_negative)

In [11]:
def transform_patients_data_into_X_y_sets(patients_data, segment_duration=1.0, info=True):
    epochs_per_patient = []
    labels = []
    
    epochs_num_per_patient = []
    for edf in raw_patients_data:
        epochs = mne.make_fixed_length_epochs(edf, duration=segment_duration, preload=True, verbose=False)
        epochs_per_patient.append(epochs)
        epochs_num_per_patient.append(len(epochs))
        
        label = get_label(edf)
        labels.extend([label for epoch in epochs])
    
    epochs = mne.concatenate_epochs(epochs_per_patient)

    if info:
        print_info(epochs_num_per_patient, labels)
        
    del epochs_num_per_patient
    gc.collect()
    
    return (epochs, np.array(labels)) # (X, y)

In [12]:
# crop_raw_data_to_equalize_duration_per_class()

In [13]:
X, y = transform_patients_data_into_X_y_sets(patients_data=raw_patients_data, segment_duration=5.0)

Not setting metadata
5771 matching events found
No baseline correction applied
0 bad epochs dropped

Epochs number per patient:  [185, 182, 182, 185, 189, 186, 182, 182, 181, 223, 183, 180, 193, 173, 169, 229, 192, 241, 178, 148, 269, 182, 237, 170, 272, 217, 227, 434]

negative:  3165
positive:  2606


In [14]:
del raw_patients_data
gc.collect()

0

In [15]:
print(len(y))
print(len(X))
print(X[0].get_data().shape)

5771
5771
(1, 19, 1250)


In [16]:
X.to_data_frame().head()

,time,condition,epoch,Fp2,F8,T4,T6,O2,Fp1,F7,...,O1,F4,C4,P4,F3,C3,P3,Fz,Cz,Pz
0,0,1,0,0.002500,0.002500,0.002500,0.00250,0.002500,0.0025,0.0025,...,0.00250,0.0025,0.002500,0.0025,0.0025,0.002500,0.00250,0.0025,0.0025,0.002500
1,4,1,0,0.002500,0.002500,0.002500,0.00250,0.002500,0.0025,0.0025,...,0.00250,0.0025,0.002500,0.0025,0.0025,0.002500,0.00250,0.0025,0.0025,0.002500
2,8,1,0,0.002500,0.002500,0.002500,0.00250,0.002500,0.0025,0.0025,...,0.00250,0.0025,0.002500,0.0025,0.0025,0.002500,0.00250,0.0025,0.0025,0.002500
3,12,1,0,0.461215,0.461215,0.308310,0.30831,0.155405,0.0025,0.0025,...,0.00250,0.0025,0.002500,0.0025,0.0025,-0.150405,-0.30331,0.0025,0.0025,-0.303310
4,16,1,0,0.461215,0.461215,0.461215,0.30831,0.155405,0.0025,0.0025,...,-0.30331,0.0025,0.155405,0.0025,0.0025,-0.150405,-0.30331,0.0025,0.0025,-0.150405


In [17]:
X[0].to_data_frame().head()

,time,condition,epoch,Fp2,F8,T4,T6,O2,Fp1,F7,...,O1,F4,C4,P4,F3,C3,P3,Fz,Cz,Pz
0,0,1,0,0.002500,0.002500,0.002500,0.00250,0.002500,0.0025,0.0025,...,0.00250,0.0025,0.002500,0.0025,0.0025,0.002500,0.00250,0.0025,0.0025,0.002500
1,4,1,0,0.002500,0.002500,0.002500,0.00250,0.002500,0.0025,0.0025,...,0.00250,0.0025,0.002500,0.0025,0.0025,0.002500,0.00250,0.0025,0.0025,0.002500
2,8,1,0,0.002500,0.002500,0.002500,0.00250,0.002500,0.0025,0.0025,...,0.00250,0.0025,0.002500,0.0025,0.0025,0.002500,0.00250,0.0025,0.0025,0.002500
3,12,1,0,0.461215,0.461215,0.308310,0.30831,0.155405,0.0025,0.0025,...,0.00250,0.0025,0.002500,0.0025,0.0025,-0.150405,-0.30331,0.0025,0.0025,-0.303310
4,16,1,0,0.461215,0.461215,0.461215,0.30831,0.155405,0.0025,0.0025,...,-0.30331,0.0025,0.155405,0.0025,0.0025,-0.150405,-0.30331,0.0025,0.0025,-0.150405


In [18]:
### Data preparation

np.set_printoptions(precision=50)

x_data = X.get_data()
print('x_data shape:', x_data.shape)

column_names = X[0].to_data_frame().columns
column_names = column_names[-19:]
print('column_names:', column_names)

epoch_num, channel_num, epoch_len = x_data.shape

x_data shape: (5771, 19, 1250)
column_names: Index(['Fp2', 'F8', 'T4', 'T6', 'O2', 'Fp1', 'F7', 'T3', 'T5', 'O1', 'F4',
       'C4', 'P4', 'F3', 'C3', 'P3', 'Fz', 'Cz', 'Pz'],
      dtype='object')


In [19]:
x_data = x_data.swapaxes(1, 2)
x_data.shape

(5771, 1250, 19)

In [20]:
input_shape = x_data[0].shape
input_shape

(1250, 19)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y, test_size=0.2, shuffle=True, random_state=1)

In [22]:
validation_data_ratio = 0.2
# input_shape = Input(shape=input_shape)
print('input_shape:', input_shape)
batch_size = 128
# n_steps = traingen.samples // BATCH_SIZE
# n_val_steps = validgen.samples // BATCH_SIZE
n_epochs = 50

input_shape: (1250, 19)


In [23]:
import random
rnd_seed = random.randint(0, 1000000)
print(rnd_seed)

976303


In [24]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, Conv1D, MaxPooling1D, MaxPooling2D, AveragePooling2D, AveragePooling1D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm

In [25]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Activation, Permute, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import SeparableConv2D, DepthwiseConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import SpatialDropout2D
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.layers import Input, Flatten
from tensorflow.keras.constraints import max_norm
from tensorflow.keras import backend as K

In [26]:
def GRU_RNN(input_shape, opt):
    model=tf.keras.Sequential()
    
    #model.add(layers.LSTM(128,
    #         input_shape=(chans, time_steps),
    #         ))
    
    # The output of GRU will be a 3D tensor of shape (batch_size, timesteps, 256)
    model.add(layers.GRU(512, 
                         input_shape=input_shape,
                         return_sequences=True))

    model.add(layers.SimpleRNN(256))
    #model.add(layers.SimpleRNN(128))
    

    model.add(Dense(32,kernel_initializer='he_uniform',activation='relu'))
    

    model.add(Dense(1,activation='sigmoid'))
    
        # Compiles the model for training.
    model.compile(optimizer=opt, 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model    

In [31]:
opt = tf.keras.optimizers.Adam(learning_rate=0.00009, 
                                beta_1=0.99,
                                beta_2=0.999,
                                epsilon=1e-07)

In [32]:
model = GRU_RNN(input_shape, opt)

In [33]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_2 (GRU)                 (None, 1250, 512)         818688    
                                                                 
 simple_rnn_2 (SimpleRNN)    (None, 256)               196864    
                                                                 
 dense_4 (Dense)             (None, 32)                8224      
                                                                 
 dense_5 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,023,809
Trainable params: 1,023,809
Non-trainable params: 0
_________________________________________________________________


In [34]:
# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='../vgg16/V1/weights/model.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=15,
                           restore_best_weights=True,
                           mode='min')

In [35]:
history = model.fit(
    x=X_train, 
    y=y_train.reshape(-1,1),
    batch_size=32,
    validation_split=0.2,
    epochs=50, 
#     callbacks=[tl_checkpoint_1, early_stop],
    callbacks=[early_stop],
    verbose=1)

Epoch 1/50
116/116 [==============================] - 1568s 14s/step - loss: 0.6904 - accuracy: 0.5466 - val_loss: 0.6873 - val_accuracy: 0.5541
Epoch 2/50
 37/116 [========>.....................] - ETA: 18:38 - loss: 0.6840 - accuracy: 0.5735

ResourceExhaustedError: Graph execution error:

Detected at node 'gradients/while_grad/gradients/while/MatMul_1_grad/MatMul_1' defined at (most recent call last):
    File "E:\Users\dt\anaconda3\envs\ml\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "E:\Users\dt\anaconda3\envs\ml\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "E:\Users\dt\anaconda3\envs\ml\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "E:\Users\dt\anaconda3\envs\ml\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "E:\Users\dt\anaconda3\envs\ml\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "E:\Users\dt\anaconda3\envs\ml\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "E:\Users\dt\anaconda3\envs\ml\lib\a

In [ ]:
# model.save('../vgg16/V1/model/')

In [ ]:
def plot_accuracy(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()


In [ ]:
plot_accuracy(history)

In [ ]:
plot_loss(history)

In [ ]:
predictions = model.predict(X_test, verbose=0)


In [ ]:
predictions = np.array([0 if x < 0.5 else 1 for x in predictions])

In [ ]:
print(len(predictions))
print(len(y_test))

In [ ]:
print(predictions[0:15])
print(y_test[0:15])

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy:", accuracy)

In [ ]:
confusion_mtx = confusion_matrix(y_test, predictions)

ax = plt.axes()
sn.heatmap(confusion_mtx, annot=True,annot_kws={"size": 25}, cmap="Blues", ax = ax, fmt='d')
ax.set_title('Test Accuracy', size=14)
plt.show()

In [43]:
def CNN(input_shape, opt, fine_tune=False, chans=19, nb_classes=2, 
             dropoutRate = 0.5, kernLength = 64, F1 = 8, 
             D = 2, F2 = 16, norm_rate = 0.25, dropoutType = 'Dropout'):
    
    if dropoutType == 'SpatialDropout1D':
        dropoutType = SpatialDropout2D
    elif dropoutType == 'Dropout':
        dropoutType = Dropout
    else:
        raise ValueError('dropoutType must be one of SpatialDropout1D '
                         'or Dropout, passed as a string.')
    
    input1 = input_shape
    
    pooling_layer1_format = 'channels_first'
    pooling_layer2_format = 'channels_last'
   

    ##################################################################
    block1       = Conv1D(F1, (kernLength), padding = 'same',
                                  # input_shape = ( chans, samples),
                                   use_bias = False)(input1)
    block1       = BatchNormalization(axis = -1)(block1)
   
    block1       = Conv1D(5, (chans), padding='same',
                          use_bias = False )(block1)
    block1       = BatchNormalization(axis = -1)(block1)
    block1       = Activation('elu')(block1)
    block1       = AveragePooling1D(pool_size=(4),  name='apl1')(block1)
    block1       = dropoutType(dropoutRate)(block1)
 
    block2       = Conv1D(5, ( 16),
                                   use_bias = False, padding = 'same' )(block1)
    block2       = BatchNormalization(axis = -1)(block2)
    block2       = Activation('elu')(block2)
    #block2       = AveragePooling1D(pool_size=(3), name='apl2')(block2)
    block2       = dropoutType(dropoutRate)(block2)
    
    
    
    block2       = Conv1D(5, ( 16),
                                   use_bias = False, padding = 'same' )(block2)
    block2       = BatchNormalization(axis = -1)(block2)
    block2       = Activation('elu')(block2)
    block2       = dropoutType(dropoutRate)(block2)
 
        
    flatten      = Flatten(name = 'flatten')(block2)
    
    dense        = Dense(1, name = 'dense',  kernel_constraint = max_norm(norm_rate))(flatten)
    
    softmax      = Activation('sigmoid', name = 'sigmoid')(dense)
    
    model = Model(inputs=input1, outputs=softmax)
    
    model.compile(optimizer=opt, 
              loss='binary_crossentropy',
              metrics=['accuracy'])
    
    return model

In [44]:
opt = keras.optimizers.Adam(learning_rate=0.0001)

In [45]:
model = CNN(input_shape, opt, chans=19, fine_tune=0)

ValueError: Layer "conv1d" expects 1 input(s), but it received 2 input tensors. Inputs received: [<tf.Tensor: shape=(), dtype=int32, numpy=19>, <tf.Tensor: shape=(), dtype=int32, numpy=1250>]

In [ ]:
model.summary()

In [ ]:
# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='../vgg16/V1/weights/model.weights.best.hdf5',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=15,
                           restore_best_weights=True,
                           mode='min')

In [ ]:
history = model.fit(
    x=X_train, 
    y=y_train.reshape(-1,1),
    batch_size=32,
    validation_split=0.2,
    epochs=50, 
#     callbacks=[tl_checkpoint_1, early_stop],
    callbacks=[early_stop],
    verbose=1)

In [ ]:
# model.save('../vgg16/V1/model/')

In [ ]:
def plot_accuracy(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [ ]:
def plot_loss(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()


In [ ]:
plot_accuracy(history)

In [ ]:
plot_loss(history)

In [ ]:
predictions = model.predict(X_test, verbose=0)


In [ ]:
predictions = np.array([0 if x < 0.5 else 1 for x in predictions])

In [ ]:
print(len(predictions))
print(len(y_test))

In [ ]:
print(predictions[0:15])
print(y_test[0:15])

In [ ]:
accuracy = accuracy_score(y_test, predictions)
print("Test Accuracy:", accuracy)

In [ ]:
confusion_mtx = confusion_matrix(y_test, predictions)

ax = plt.axes()
sn.heatmap(confusion_mtx, annot=True,annot_kws={"size": 25}, cmap="Blues", ax = ax, fmt='d')
ax.set_title('Test Accuracy', size=14)
plt.show()

## TESTS

In [ ]:
X_train.shape


In [ ]:
X_train[0]

In [ ]:
X_train[0].transpose().shape

In [ ]:
X_train[0].shape

In [ ]:
X_train.reshape(3595, 1250, 19)

In [ ]:
X_train.transpose()[0].shape

In [ ]:
X_train[0].transpose()

In [ ]:
X_train.swapaxes(1, 2)

In [ ]:
X_train = X_train.swapaxes(1, 2).shape